In [9]:
import logging
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
from matplotlib import pyplot as plt
import numpy as np
import math
import json
import os
from copy import deepcopy, copy
from nim_utils import *

logging.basicConfig(
    format="[%(asctime)s] %(levelname)s: %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

In [10]:
class Agent():
    def __init__(self, nim, alpha = 0.1, random_factor = 0.2) -> None:
        self.state_history = []
        self.alpha = alpha
        self.random_factor = random_factor
        # Q table, initially empty
        self.Q = {}
        self.init_q(nim)

    def init_q(self, nim) -> dict:
        result = []
        def generate_states(rows, current_state):
            if len(current_state) == len(rows):
                # We have generated a valid state 
                result.append(current_state)
                return
            # Generate all possible states for the current row
            for i in range(rows[len(current_state)] + 1):
                new_state = current_state + [i]
                generate_states(rows, new_state)
        
        generate_states(nim.rows, [])
        for state in result:
            x = Nim(len(nim.rows), state=state)
            for action in x.possible_moves():
                self.Q[(x, action)] = np.random.uniform(low=1.0, high=0.1)     

    def choose_action(self, state) -> Nimply:
        maxG = -10e15
        next_move = None
        possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
        if np.random.random() < self.random_factor:
            next_move = random.choice(possible_moves) 
        else:
            for action in possible_moves:
                # print(f"Action: {action}")
                new_state = deepcopy(state)
                new_state.nimming(action)
                # print(f"Check {new_state}")
                if self.G[new_state] >= maxG:
                    next_move = action
                    maxG = self.G[new_state]
        return next_move

    def update_state_history(self, state, reward):
        self.state_history.append((state, reward))

    def learn(self):
        target = 0
        for prev, reward in reversed(self.state_history):
            self.G[prev] = (1 - self.alpha)*self.G[prev] + self.alpha * (target - self.G[prev])
            target += reward
        self.state_history = []
        self.random_factor -= 10e-5  # decrease random factor each episode of play

    def get_strategy(self) -> Callable:
        def agent_strategy(state: Nim) -> Nimply:
            action = self.choose_action(state)
            # print(f"Choosen action: {action}")
            return Nimply(action[0], action[1])
        return agent_strategy

In [11]:
def evaluate_against(strategy: Callable, against: Callable, NIM_SIZE: int, NUM_MATCHES = 100) -> float:
    opponent = (strategy, against)
    won = 0
    for _ in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            # logging.debug(nim)
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1: # winner is the zero
            won += 1
        # logging.debug(f"player {1 - player} has won.")
    return won / NUM_MATCHES

### Reinforcement learning strategies


In [12]:
# Costants
NUM_EPOCHS = 100000
NIM_SIZE_learn = 2
OPPONENTS = [aggressive] * 5 + [optimal_strategy]

# Create the agent
game = Nim(NIM_SIZE_learn)
print(f"size = {len(game.rows)}")
agent = Agent(game, alpha=0.99, random_factor=.1)

states = agent.Q

size = 2


In [17]:
for state in states:
    print(f"{state[0]}, {state[1]}: {states[state]}")

<0 1>, (1, 1): 0.5325767768100627
<0 2>, (1, 1): 0.6515700550948612
<0 2>, (1, 2): 0.17030622417935637
<0 3>, (1, 1): 0.5428793574177256
<0 3>, (1, 2): 0.92818945473787
<0 3>, (1, 3): 0.9710447536309582
<1 0>, (0, 1): 0.7805588776908537
<1 1>, (0, 1): 0.6069205158619628
<1 1>, (1, 1): 0.2701561610913368
<1 2>, (0, 1): 0.565036555603934
<1 2>, (1, 1): 0.4197333442194956
<1 2>, (1, 2): 0.207675511445776
<1 3>, (0, 1): 0.48511230833451613
<1 3>, (1, 1): 0.5269384803095291
<1 3>, (1, 2): 0.22688082846673074
<1 3>, (1, 3): 0.45216064351913143


In [14]:
# for state in states:
#     print(f"{state}: {states[state]}")
win_logs = []
scores = []
for i in range(NUM_EPOCHS):
    # logging.debug(f"Starting the game #{i}")
    current_game = Nim(NIM_SIZE_learn)
    turn = True
    OPPONENT = random.choice(OPPONENTS)
    while current_game:
        if turn:
            # my agent turn
            action = agent.choose_action(current_game)
            # print(f"Action: {action}")
            current_game.nimming(action)
            state, reward = current_game.get_state_and_reward()
            # print(f"{state} -> {reward}")
            agent.update_state_history(state, reward)
        else:
            action = OPPONENT(current_game)
            current_game.nimming(action)
        turn = not turn
    winner = int(not turn)
    win_logs.append(winner)
    # logging.debug(f"The game ended. Player {winner} wins")
    agent.learn()
    if i % 300 == 0:
        score = evaluate_against(agent.get_strategy(), aggressive)
        scores.append(score)
        print(f"#{i}: {score}")


agent_strategy = agent.get_strategy()


AttributeError: 'Agent' object has no attribute 'G'

In [ ]:
plt.scatter(range(len(win_logs)), win_logs, marker='o', linewidths=.2, edgecolors=None)
plt.show()

In [ ]:
plt.plot(range(len(scores)), scores)
plt.show()

In [ ]:
evaluate_against(agent_strategy, aggressive)

